In [ ]:
import numpy as np
import os, random
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf
plt.rcParams["figure.figsize"] = [15, 15]


PTH ="/kaggle/input/shopee-code-league-2020-product-detection/resized/{}"
PTH_TRAIN = PTH.format("train")
PTH_TEST = PTH.format("test")

train =  tf.keras.preprocessing.image_dataset_from_directory(
    PTH_TRAIN,
    validation_split = .2,
    subset = "training",
    seed = 1,
    labels     = "inferred",
    label_mode = "int",
    image_size = (299, 299)
)

test =  tf.keras.preprocessing.image_dataset_from_directory(
    PTH_TRAIN,
    validation_split = .2,
    subset = "validation",
    seed = 1,
    labels     = "inferred",
    label_mode = "int",
    image_size = (299, 299)
)

class Preprocess(tf.keras.layers.Layer):
    def __init__(self):
        super(Preprocess, self).__init__()
#         self.flip = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical")
#         self.rotate = tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)
    
    def call(self, X):
#         X = self.flip(X)
#         X = self.rotate(X)
        return tf.keras.applications.efficientnet.preprocess_input(X)

basemodel = tf.keras.applications.EfficientNetB6(
    include_top = False , weights='imagenet', input_tensor=None,
    input_shape = None, pooling=None,)

basemodel.trainable = False

net = tf.keras.Sequential([
    Preprocess(),
    basemodel,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(42, activation = "softmax")
])


net.compile(
    optimizer="adam",
    loss = "sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

with tf.device("/device:GPU:0"):
    history = net.fit(
        train,
        validation_data = test, 
        epochs=10)
    

# Fine tuning the 
basemodel.trainable = True
net.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss = "sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

with tf.device("/device:GPU:0"):
    history = net.fit(
        train,
        validation_data = test, 
        epochs=5)